# Processing and analyzing MS2 movies with Python

This notebook demos some of the code I've been using to process and analyze MS2 movies, specifically confocal movies that simultaneously image the transcription of a gene vis MCP binding to nascent transcripts and the nuclear distribution of a protein factor of interest. The code is ever-evolving, and some aspects of it are admittedly not particularly good. Still, it provides a decent starting point for analyzing this kind of data with Python/Jupyter. For anyone who wants to use it, I encourage you to steal anything that might be useful and to improve anything you see that needs improvement (and please share!).

### Step 0: Import packages

I use a combination of publicly available packages as well as an expanding library of functions I have written myself. The first step is to import the public packages by running the cell below. I don't have a list of dependencies yet, so my recommendation would be to execute the cell below (and the next), check the error messages, and install packages to fill in what's missing. I don't think I use any exotic packages...

In [ ]:
# Import public packages.
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi 
from importlib import reload
import pickle
import czifile
%matplotlib inline

In addition to these packages, I've written a lot of functions myself. I've packaged these functions into a module called __imagep__. I wrote it to be for personal use, but I have diligently documented and commented this code with a goal of making it at least *somewhat* usable by others.

This module actually requires a second module, __fitting.py__, which should be available in the github directory with this notebook. To use them, put them in a folder on your computer and assign the path to that folder to the __folder__ variable in the first line of the cell below, then execute the cell and fix any dependency problems.

In [ ]:
folder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/bin'

import sys
sys.path.append(folder)
import imagep as imp
import fitting as fit
from imagep import viewer, save_pickle, load_pickle, box_spots, read_concat_5dczi

### 1. Read in czi files

The first step is to read in the image files. Because these movies (ideally) last for tens of minutes, it is sometimes necessary to stop imaging to refocus in order to compensate for microscope drift in the Z-axis. This results in datasets that are broken up into multiple distinct movies, each of which is taken at an adjusted Z position. These re-focusing events are specifically handled later (at the step of connecting detected spots between time frames). For now, we combine all the movie files into a single image stack (a 5D numpy ndarray), and store the positions where movies are joined in __frames__.

In [ ]:
# Read in czi file.
nucchannel = 0
spotchannel = 1

files = [
    '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/20200811-em2-zsmv-04.czi',
    '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/20200811-em2-zsmv-05.czi'
]

stack, frames = read_concat_5dczi(files)

### 2. View movie

A very basic but important thing is to take a look at your movie. __viewer__ is an interactive tool that is very useful tool for all these analyses. It's pretty basic, but gives you a quick and easy (and so far, stable!) way to look at these data. A couple tips: 1) if you click on a slider to select it, you can move them with the arrow keys, 2) you can click on the numbers and directly enter desired values with the keyboard. These two tips can help avoid the annoying issue of using a track pad to slide juuuust the right amount.

In [ ]:
viewer(stack)

__viewer__ will do a decent job if you just give it an image stack, but you can supply more detail if you want, changing the size of the figure, the labels of the sliders, the default colors, or displaying a maximum z projection. All the functions in __imagep__ have complete doc strings, so if you want to know details about a function, use the ? notation:

In [ ]:
?viewer

In [ ]:
# Example of some things you can do with viewer:
viewer(stack, figsize=16, zmax=True, color="viridis")

## Part 1: Segmenting nuclei

Now that we've loaded and looked at out movie, our first serious task is to segment nuclei (figure out where the nuclei are and give each one a unique label so we can tell them apart). This actually isn't strictly necessary but ends up being quite useful. For these types of movies (where the nuclei form a monolayer with very similar locations in Z), I am taking a shortcut and just doing 2D segmentation on a maximum projection. 3D segmentation is definitely the proper way to go, but it's more time-consuming and a little pricklier (in my hands), so for the purpose of just assigning spots to nuclei, this works fine.

#### Disclosure: this nuclear segmentation routine is only OK at the moment. I got it most of the way there, but I haven't used nuclear masks much in my analysis so I haven't really tweaked it. I still lose nuclei for unclear reasons...I plan to fix this in the future


### 3. Determine parameters for segmenting nuclei.

Unfortunately, it's not trivial to come up with a generic nuclei segmentation algorithm. For my projects, I would like to be able to segment using a variety of nuclear labels (different fluorescently-tagged nuclear proteins). These proteins do not all look the same, and that means that some of them require slightly different treatments to get good segmentation. Specifically, proteins that form bright nuclear substructures can be difficult to segment by the same methods that work for more uniform labels (e.g. histones).

What I have tried to do to ease some of the difficulty of this is to separate the tasks some. The function __segment_nuclei_4dstacks__ is a useful wrapper that handles all the work of applying a segmentation function that works on single 3D frames to an entire 4d stack. __connect_nuclei__ then connects nuclei between frames. That means that all you have to do is write a function that can segment nuclei from a single 3D image of your data (this function gets supplied as an argument to __segment_nuclei_4dstacks__), and all the other stuff gets handled so you don't have to copy and paste all the constant parts ever time you work with a new protein.

I have written a function specially for rpb1 data, called __segment_nuclei3D_monolayer_rpb1__. For uniform nuclear labels like histones, __segment_nuclei3D_monolayer__ works well.

I start by running the segmentation on the first few frames of the movie to test the "knobs" I can turn for the segmentation function, which include filters for the max and min sizes of nuclei and their circularity. You can use the viewer to assess the success of the segmentation, and tweak the parameters until you're happy with it.

##### Note: Sometimes segmentation will fail and it can be hard to figure out where the problem is happening. I didn't include it here (for simplification reasons), but the 3D segmentation functions have "display" modes that will display intermediates in the segmentation algorithm. A useful way to troubleshoot is often to try segmenting a single frame with this mode on and see where the breakdown is occurring.

In [ ]:
seed_window = (75,75) # Expected minimum distance in x,y pixels between centers of nuclei.
size_max = 6000 # Maximum size of final nuclei objects, in pixels.
size_min = 100 # Minimum size
circularity_min = 0.5 # Minimum 'circularity' of final nuclei

seg_func = imp.segment_nuclei3D_monolayer_rpb1

nucmask_test = imp.segment_nuclei_4dstack(stack[nucchannel,0:5], seg_func, seed_window=seed_window, size_max=size_max, circularity_min=circularity_min)
viewer(nucmask_test, figsize=12)

### 4. Perform initial segmentation and connect nuclei

Once you've found some good parameters using the first few frames, run the segmentation on the entire dataset, and also connect nuclei to each other between frames using the __connect_nuclei__ function.

In [ ]:
nucmask_init = imp.segment_nuclei_4dstack(stack[nucchannel], imp.segment_nuclei3D_monolayer_rpb1, seed_window=seed_window, size_max=size_max, circularity_min=circularity_min)

max_frames_skipped=2
nucmask_conn = imp.connect_nuclei(nucmask_init, max_frames_skipped=max_frames_skipped)

#### Optional: view connected segmentation

In [ ]:
viewer(nucmask_conn, figsize=15)

### 5. Interpolate to fill in gaps

Particularly for Rpb1, I occasionally have nuclei drop out of frames. This function corrects that by interpolation, which admittedly doesn't work amazingly well at the moment. It's a work in progress...

In [ ]:
nucmask = imp.interpolate_nuclear_mask(nucmask_conn, 3)

#### View final nuclear segmentation

In [ ]:
viewer(nucmask, figsize=12)

As I said — it's not perfect. It mostly gets your nuclei, but you get some weird things like a nucleus changing ID for no reason. [shrug emoji]

## Part 2: Detect MS2 spots

The next all-important task is to find MS2 spots. The strategy I use is to fit a 3D gaussian function to candidate spots. The resulting fits give information on the shape, size, and intensity of candidate spots, and this information can be used to filter for "real" spots. It's not a perfect approach, but it does quite well for high signal-to-noise movies.

## Interlude: A few notes on fitting

What does it mean to "fit a 3D gaussian" to a spot? First, a 3D gaussian is a normal distribution in 3D. For microscopy data, a 3D gaussian would be brightest in the center pixel and then the brightness would decay exponentially in all directions as you move away from the center. I've made a 3d gaussian function in the center of this viewing window:

In [ ]:
x, y, z = np.indices([9,100,100])
# p = (center_z, center_x, center_y, height, width_z, width_x, width_y)
p = (4, 50, 50, 255, 5, 15, 15)
f = fit.gaussian3d(*p)
g = f(x, y, z)
viewer(g, 6, color="viridis")

So we are saying that we think our spots will be modeled reasonably well by a 3D gaussian function. An individual 3d gaussian distribution is characterized by seven parameters: the intensity (sometimes called 'height' in an analogy to a 1D gaussian distribution), the location of the center of the distribution in each dimension (3 parameters), and the width (standard deviation) of the distribution in each dimension. In mathier notation, the 3D gaussian is a function:

3D gaussian = f(height, center_z, center_x, center_y, width_z, width_x, width_y)

So each unique combination of those 7 parameters gives a unique distribution. When we "fit" a gaussian to a spot, all we are saying is "find the combination of these 7 parameters that gives an intensity distribution that most closely matches our real data". 

For a graphical representation, imagine we are playing a game (ok it's the shittiest game in the universe, but it's the covid era and we'll try anything to stay occupied). You have a control panel with 7 knobs controlling the 7 parameters above, and you start with the centered distribution above. Then you get a new picture of a random spot, and you have to adjust the knobs to make them match:

##### You can 'play' the game if you'd like by tweaking the parameters listed at the top of the cell below to change the gaussian on the left until it matches the one on the right. You can also just follow along conceptually. Really depends how bored you've gotten at home.

In [ ]:
z_center = 2
x_center = 36
y_center = 36
intensity = 200
width_x = 25
width_y = 25
width_z = 5

p2 = (z_center, x_center, y_center, intensity, width_z, width_x, width_y)
f2 = fit.gaussian3d(*p2)
g2 = f2(x, y, z)
viewer([g2,g], 6, color="viridis")

You notice that the new spot is off-centered, so you'd probably start by moving the center coordinates until the guy on the left is centered on the one on the right. Then you notice the new spot is dimmer, so you roll down the intensity until they seem equally bright. Then you notice the spot on the right is narrower, so you start dialing down the widths. You fiddle around with the knobs for a while until you've made the spot on the left look as close to the spot on the right, and you're done. That's it: you've fit a gaussian to a spot.

All we are doing is having a computer do the same thing. We define an error function (here: the sum of the squares of the differences in pixel values over the window), and ask the computer to test parameter combinations in an effort to minimize that error function. I use scipy's __optimize__ module for fitting, specifically its __least_squares__ function. This function takes an error function, parameters, and optional bounds for the parameters, and finds parameters that minimize the squared error function. If you're interested, you can dig into all this in the __fit_ms2__ function and the functions therein (in __fitting.py__).

### 6. Perform gaussian fitting on MS2 channel

I do my fitting using the __fit_ms2__ function on the MS2 channel. This function generates a candidate list of spots by identifying local maxima within windows of a user-defined size. For each candidate spot (a point in z,x,y within the image), it fits a 3D gaussian function to the region of the image constituting a small 3D box centered on that point. The resulting fit parameters (intensity, z-center, x-center, y-center...) are stored in the __fits__ object. You can view the raw fits using __quickview_ms2__ which draws boxes around all the locations in fits (which are all the local maxima).

In [ ]:
min_distances = (14, 20, 20)
fitwindow_rad_xy = 10
fitwindow_rad_z = 2
sigma_small = 1

fits = imp.fit_ms2(stack[spotchannel], min_distances=min_distances, fitwindow_rad_xy=fitwindow_rad_xy, sigma_small=sigma_small)

imp.quickview_ms2(stack, fits, 1)

### 7. Filter fits

So it's obvious that many of our local maxima are not real spots. The next step is to filter the candidates based on the fit parameters. I used a simple strategy of filtering on 'peakiness', which is the equivalent of of the ratio of the height to width (sigma) of a 1D gaussian. I don't currently have any rigorous way of selecting peak thresholds — I'm just testing out different values until it seems to get real spots. Something worth considering is that a few false positives in a frame isn't terrible, because these "orphan" spots will fail to connect to spots in adjacent frames and can be filtered out that way (ultimately by the shortness of their trajectories).

In [ ]:
peakiness = 4

fits_filtered = imp.filter_ms2fits(stack, fits, spotchannel, peakiness)

imp.quickview_ms2(stack, fits_filtered, spotchannel)

### 8. Determine Z adjustments (if unknown)

Now that we have detected spots in individual frames, we need to connect the frames so that we can follow individual spots in time. The connection is done by defining a maximum "jump length" that a spot is allowed to move between frames. Spots in adjacent frames are linked if they are within this jump length, with some tolerance for frame dropout (a spot can be missing from one or more frames and be connected across them).

Here is where we need to correct for the re-focusing breaks. If uncorrected, the spots will all appear to "jump" in Z at the junction, which could potentially cause missed connections.

If you already know the corrections (e.g., you carefully moved the focal plane 2 slices), you can skip this. If not, you can use the viewer to manually determine corrections). The way to do this is to call the viewer twice (see below), once on the junction frame and once on the frame before it. Then adjust the Z sliders until you are convinced both are showing the same Z position. The correct is then equal to the difference of the z slice in the junction frame minus the z slice of the previous frame.

In [ ]:
# See the frames that represent junctions between re-focused movies.
print(frames)

In [ ]:
# Set one viewer to one of the frames in frames, the other to the frame before it (n and n-1).
viewer(stack[nucchannel, 38], figsize=10)
viewer(stack[nucchannel, 39], figsize=10)

For this movie, I decided slice 5 in the top and slice 3 in the bottom show about the same Z position. This gives me a correction of 5 - 3 = 2. That goes in __z_corrs__. There's only one junction here, but if there were multiple, the corrections would correspond in position to the frames. For example, a junction at frame 10 with a correction of 2 and a junction at frame 21 with a correction of 1 would have __frames__ = [10, 21] and __z_corrs__ = [2,1].

In [ ]:
# Set focus correction for each junction.
# For each junction, supply the number that must be added to the *later* frame to match positions to the earlier frame.

z_corrs = [2]

### 9. Connect spots from fits

Now that we have the corrections, we can connect our spots in time, supplying a maximum allowable jump length between frames and maximum number of allowable dropped frames. As with nuclear segmentation, I try it out on the first few frames and make sure it looks reasonable. Here, I filter for spot duration (number of frames the spot is found in) as a quick way to assess well-connected spots.

__*Run on a subset of the data to tweak parameters*__

In [ ]:
max_frame_gap = 2
max_jump = 18
scale_xy = 1
scale_z = 1

# Test on first n frames.
n = 6
min_len = 5

test_spots_connected = imp.connect_ms2_fits_focuscorrect(fits_filtered[0:n], [0], [0], nucmask, 
    max_frame_gap=max_frame_gap, max_jump=max_jump, scale_xy=scale_xy, scale_z=scale_z)

test_spots_connected = imp.filter_spot_duration(test_spots_connected, min_len)

imp.quickview_ms2(stack, test_spots_connected, spotchannel)

Once I'm happy with the parameters for the first few frames, I run it for the whole dataset:

In [ ]:
# Connect all frames.
spots_connected = imp.connect_ms2_fits_focuscorrect(fits_filtered, [0], [0], nucmask, 
    max_frame_gap=max_frame_gap, max_jump=max_jump, scale_xy=scale_xy, scale_z=scale_z)

__*Optional: filter for minimum trajectory length*__

A characteristic of "real" MS2 spots is that they should persist over time. Besides, for most analyses, spots have to last for at least some number of frames to be useful. For that reason, I typically filter for spots that have some minimum duration. This often cleans up the spot detection.

In [ ]:
min_len = 10

# Filter for trajectory length.
spot_data = imp.filter_spot_duration(spots_connected, min_len)

As always, it's good to check the results visually and see if they make sense (we'll do some more QC on detections later).

In [ ]:
# Check the results.
imp.quickview_ms2(stack, spot_data, channel = spotchannel, MAX=True)

### 10. Add integrations of data around spots

The data we ultimately want to get out of all this tracking is (primarily) the intensity of the spots. To do this, we integrate (sum up or average) the fluorescence signal in some volume around the center of the spots. The __add_volume_mean__ function defines an ellipsoid (defined by a radius in the lateral [xy] and axial [z] dimensions) around each spot's center and takes the mean intensity of the pixels in this volume. The results are added an an additional column to the __spot_data__ object.

In [ ]:
ij_rad = 6
z_rad = 1.1
ij_scale = 1
z_scale = 1

spot_data2 = imp.add_volume_mean(spot_data, stack, spotchannel, ij_rad, z_rad, ij_scale, z_scale)

A related way to measure spot intensity is to do a mathematical integration of the gaussian fits for each spot. This is the 3D equivalent to integrating the "area under the curve" of a 1D gaussian. If the spots are indeed well-described by Gaussian functions, the integrated volumes and gaussian integrations should be similar (we'll see how similar they are later).

The function __add_gaussian__integration__ does the integrating for each spot based on its fit parameters, and adds the results as a column to __spot_data__.

In [ ]:
wlength_ij = 13
wlength_z = 3
reload(imp)
spot_data3 = imp.add_gaussian_integration(spot_data2, wlength_ij, wlength_z)

Finally, a major goal of these experiments is to simultaneously monitor the protein signal at the locus. To do this, I just use the __add_volume_mean__ function again, this time on the protein channel. By default I use the same sized ellipsoid for the protein and MS2 channels, but there's no reason this needs to be true (the size of the "effect" of the protein need not coincide with the size of the region occupied by nascent transcripts).

In [ ]:
spot_data4 = imp.add_volume_mean(spot_data3, stack, nucchannel, ij_rad, z_rad, ij_scale, z_scale)

### 11. Package into *movie* object and save

We've produced a lot of things worth keeping track of. If you're working on analyzing a movie, you might want to have ready access to the original data (the image stack), the nuclear mask, the fit data, the spot locations, the volume integrations in protein and MS2 channels...and maybe other forms of data that I haven't thought of or worked out yet (burst starts and ends?). I often find myself working on multiple movies at once, so this can become a nightmare quickly. Naturally, classes come to the rescue.

I created a very simple __movie__ class that is mostly just a container. It stores the principal outputs of the processing functions above (also the raw fits, which are computationally intensive to generate, allowing you to re-filter for tweaking spot detection without re-running the fitting). The only thing it *does* in addition to storing things is to generate some useful pandas dataframes for the integrated signals (intvol, intfit, and prot).

The attributes of the movie class (so far) are:

__stack__ - original stack  
__nucmask__ - 4d nuclear mask  
__fits__ - un-filtered gaussian fits  
__spot_data__ - locations, fit parameters, and MS2 and protein integrations for called spots  
__intvol__ - panda dataframe of MS2 signal (intensity of integrated volumes in original data)  
__intfit__ - panda dataframe of MS2 signal (mathematical integration of gaussian fits)  
__prot__ - panda dataframe of protein signal (intensity of integrated volumes in original data)  

Once everything is packaged into a __movie__ object, I save it via the pickle (I wrote a small wrapper called __save_pickle__ that just streamlines the built-in syntax, but it's not hard). Pickle is how Python does object serialization, which lets you store an object *and its current state* on your hard drive to retrieve later.

In [ ]:
mv = imp.movie(stack, nucmask, fits, spot_data4)
mv_filename = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/20180811-rpb1-eve-em2-zsmv-04-05.pkl'
save_pickle(mv, mv_filename)

Here is an example of the panda dataframes included in __movie__ objects:

In [ ]:
mv.intvol

### 12. Do some QC on tracking.

Obviously, we want to do some quality control to make sure that the steps we took above actually worked and produced reasonable results. From personal and vicarious experience, tracking errors can create artifacts that look like plausible signals.

A useful thing to look at is trajectory lengths (how many frames each spot appears in). This example movie has a number of spots that turn off as the stripe moves, so we don't expect all spots to last the whole movie. Still, it's useful to look at.

In [ ]:
lens = np.sort(np.count_nonzero(~np.isnan(mv.intvol), axis=0))
plt.hist(lens, bins = np.arange(0,len(mv.intvol)+1));
print(lens)

Another thing to look at is how well the two methods of signal integration (summing the raw signal vs. integrating the fitted gaussian function) match either other. These can potentially differ for reasons that don't reflect errors, but in my experience with these data, they tend to match very well and poor matches suggest problems in spot detection. These plots also give you a chance to look at the shape of individual spot intensity traces, which can be diagnostic once you get used to looking at these.

In [ ]:
# Plot integrated volumes and gaussian fits together.
num_to_plot=len(mv.spot_data)
def test(x):
    plt.plot(mv.spot_data[x][:,9] / np.mean(mv.spot_data[x][:,9]), marker="o") # volume integration in blue.
    plt.plot(mv.spot_data[x][:,10] / np.mean(mv.spot_data[x][:,10]),marker="o") # gaussian integration in orange.
    plt.title(x)
imp.plot_ps(test, range(1,num_to_plot+1))

Probably the most useful readout is to look at the localization of individual spots. The following cell displays a time-averaged (average signal for all frames containing the spot) view of the signal around each spot from an xy and xz perspective, allowing you to visually assess the accuracy of lateral and axial localization.

In [ ]:
sm = imp.spot_movies(mv.stack, mv.spot_data, 1,17, fill=0, view=False)

for x in range(1,len(sm)):
    xy = sm[x].mean(axis=(0,1))
    xz = sm[x].mean(axis=(0,2))
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(xy)
    ax[0].set_title(str(x) + ' - XY')
    ax[1].imshow(xz)
    ax[1].set_title(str(x) + ' - XZ')


### Some tools to visualize these data

The above steps collapse the data into some useful tables, but it's always good to be cautious about the outputs of that processing. To help with this, I try to make tools which let me visualize the data, both for exploration purposes and also as a way to check whether I believe the numbers spit out by the processing steps above. I haven't done much yet, but I'll demo a couple examples here.

First, an obvious thing is just looking at a spot in the original movie. __quickview_ms2__ can accept a spot ID and just box that spot, letting you see where the computer thinks that spot is in your movie. You can pick up tracking errors this way, as well as possibly match measured intensity changes to what you see in the raw movies.

In [ ]:
spot_to_view = 2
imp.quickview_ms2(mv.stack, mv.spot_data, spotchannel, spot_to_view, MAX=True)

This viewer can be used on either channel, so you can also track the location of a spot against the protein channel:

In [ ]:
spot_to_view = 2
imp.quickview_ms2(mv.stack, mv.spot_data, nucchannel, spot_to_view, MAX=True, spotmode=True)

The above are max projections, but you can also view the non-projected version and track the spots through Z-slices. This gets pretty cumbersome. An alternative is to center the viewer on the spot, and that's what __spot_movies__ does. For each spot, it makes a mini movie of the data in a window centered on the spot. You can use the old __viewer__ function to look at the results. The first dimension ('i') is the spot ID (there is no 0 spot so it's blank), and for each spot you can scroll through relative z-slices and time frames. This serves as a useful way to assess both spot tracking and intensity measurements and also to just get your eyeballs on the data and hopefully learn some things.

In [ ]:
sm = imp.spot_movies(mv.stack, mv.spot_data, spotchannel, 31, fill=0, view=False)

viewer(sm, figsize=6, order='itzxy', color="viridis")

__viewer__ can also take multiple image stacks (supplied as a list-like object), so you can, for example, visualize the spot movies for the MS2 and protein channels at the same time:

In [ ]:
smp = imp.spot_movies(mv.stack, mv.spot_data, nucchannel, 41, fill=0, view=False)

imp.viewer([smp, sm], figsize=6, order='itzxy', color="magma")

## Fin.